In [2]:
import polars as pl
from polars import col

In [34]:
vuelos_lf = (
    pl.scan_parquet('/content/vuelos.parquet')

)
tarjetas_lz = pl.LazyFrame({'tarjeta': ['bronce','plata','oro']})
descuentos_lz = pl.LazyFrame({'descuento': [10,20,30]})

In [35]:
vuelos_lf

In [36]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
vuelos_lf.bottom_k(20,by='DEPARTURE_TIME', descending=True).collect()  ## null_last no funciona ordenamos por departue time y descdenciente

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
i32,i32,i32,i32,str,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i32,i32,i32,i32,i32
2015,2,1,7,"""AA""",2200,"""N3BSAA""","""MIA""","""MCO""",2349,2400,11,19,19,60,68,40,192,59,9,49,108,19,0,0,null,8,0,11,0,0
2015,1,2,5,"""B6""",333,"""N529JB""","""MCO""","""SJU""",2355,2400,5,17,17,161,161,140,1189,337,4,336,341,5,0,0,null,null,null,null,null,null
2015,1,2,5,"""B6""",745,"""N556JB""","""JFK""","""PSE""",2359,2400,1,21,21,227,219,187,1617,428,11,446,439,-7,0,0,null,null,null,null,null,null
2015,1,3,6,"""EV""",4629,"""N21130""","""RIC""","""IAH""",1759,2400,361,10,10,206,216,201,1157,231,5,2025,236,371,0,0,null,10,0,361,0,0
2015,1,4,7,"""EV""",4513,"""N14993""","""CLE""","""ORD""",1950,2400,250,11,11,92,87,66,315,17,10,2022,27,245,0,0,null,0,0,0,0,245
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2015,1,11,7,"""US""",1774,"""N123UW""","""CLT""","""DTW""",2219,2400,101,18,18,99,94,69,500,127,7,2358,134,96,0,0,null,0,0,96,0,0
2015,1,11,7,"""NK""",371,"""N632NK""","""FLL""","""BQN""",2355,2400,5,12,12,141,138,120,983,312,6,316,318,2,0,0,null,null,null,null,null,null
2015,1,12,1,"""AA""",2400,"""N3FMAA""","""LAX""","""DFW""",5,2400,-5,31,31,169,178,143,1235,454,4,454,458,4,0,0,null,null,null,null,null,null


## tomamos la variable de vuelos lazyframe, agrupamos por aerolinea, agregaciones una pór una y renombrar nombre de columna

In [38]:
vuelos_lf.group_by('AIRLINE').agg(
    pl.col('AIR_TIME').mean().name.suffix('_MEAN'), ## columna tiempo en el aire y nueva columna y un sufijo
     pl.col('AIR_TIME').count().alias('COUNT_AIR_TIME'),## englobo la agregacion columna tiempo en el aire, conteo y renombrarla
     SUM_DELAY=pl.col('DEPARTURE_DELAY').sum()  ## nombre de la columna = AGREGACI0N suma de retrasos
).collect() ## para ejecutar y obtener el resultado

AIRLINE,AIR_TIME_MEAN,COUNT_AIR_TIME,SUM_DELAY
str,f64,u32,i32
"""DL""",115.665353,870275,6427294
"""UA""",164.835886,507762,7355348
"""MQ""",66.97963,278791,2837908
"""EV""",72.763842,554752,4857338
"""US""",125.774234,194223,1196447
…,…,…,…
"""B6""",143.858492,262042,3026467
"""VX""",182.630796,61248,553852
"""AS""",157.844784,171439,306997


In [47]:
vuelos_lf.drop('YEAR','MONTH','DAY').collect()  ## eliminar las columnas con .drop

DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
i32,str,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i32,i32,i32,i32,i32
4,"""AS""",98,"""N407AS""","""ANC""","""SEA""",5,2354,-11,21,15,205,194,169,1448,404,4,430,408,-22,0,0,null,null,null,null,null,null
4,"""AA""",2336,"""N3KUAA""","""LAX""","""PBI""",10,2,-8,12,14,280,279,263,2330,737,4,750,741,-9,0,0,null,null,null,null,null,null
4,"""US""",840,"""N171US""","""SFO""","""CLT""",20,18,-2,16,34,286,293,266,2296,800,11,806,811,5,0,0,null,null,null,null,null,null
4,"""AA""",258,"""N3HYAA""","""LAX""","""MIA""",20,15,-5,15,30,285,281,258,2342,748,8,805,756,-9,0,0,null,null,null,null,null,null
4,"""AS""",135,"""N527AS""","""SEA""","""ANC""",25,24,-1,11,35,235,215,199,1448,254,5,320,259,-21,0,0,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4,"""B6""",688,"""N657JB""","""LAX""","""BOS""",2359,2355,-4,22,17,320,298,272,2611,749,4,819,753,-26,0,0,null,null,null,null,null,null
4,"""B6""",745,"""N828JB""","""JFK""","""PSE""",2359,2355,-4,17,12,227,215,195,1617,427,3,446,430,-16,0,0,null,null,null,null,null,null
4,"""B6""",1503,"""N913JB""","""JFK""","""SJU""",2359,2350,-9,17,7,221,222,197,1598,424,8,440,432,-8,0,0,null,null,null,null,null,null


## podemos fitrar un lazy frame

In [40]:
vuelos_lf.filter((pl.col('MONTH') > 6) & (pl.col('DEPARTURE_DELAY')> 0)).select(
  pl.col('MONTH'),
  pl.col('DEPARTURE_DELAY')
).collect()


MONTH,DEPARTURE_DELAY
i32,i32
7,38
7,48
7,9
7,7
7,20
…,…
12,159
12,16
12,7


In [41]:
descuentos_lz.collect()

descuento
i64
10
20
30


In [50]:
tarjetas_lz.with_context(descuentos_lz).select(  ## with context permite acceder a columnas de contexto externo al grafico de calculo accedea columnas de dataframe que no sean este
    pl.col('tarjeta'),  ### queremos crear un nuevo lazy frame con columnas diferentes, en una misma consulta sin necesidad de join
    pl.col('descuento')
).collect()

tarjeta,descuento
str,i64
"""bronce""",10
"""plata""",20
"""oro""",30


In [51]:
vuelos_lf.slice(1,10).collect()  ## nos permite obtener una porcion o un slice tenemos que proporcionarle de donde a donde fila 1 a fila 10

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
i32,i32,i32,i32,str,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i32,i32,i32,i32,i32
2015,1,1,4,"""AA""",2336,"""N3KUAA""","""LAX""","""PBI""",10,2,-8,12,14,280,279,263,2330,737,4,750,741,-9,0,0,null,null,null,null,null,null
2015,1,1,4,"""US""",840,"""N171US""","""SFO""","""CLT""",20,18,-2,16,34,286,293,266,2296,800,11,806,811,5,0,0,null,null,null,null,null,null
2015,1,1,4,"""AA""",258,"""N3HYAA""","""LAX""","""MIA""",20,15,-5,15,30,285,281,258,2342,748,8,805,756,-9,0,0,null,null,null,null,null,null
2015,1,1,4,"""AS""",135,"""N527AS""","""SEA""","""ANC""",25,24,-1,11,35,235,215,199,1448,254,5,320,259,-21,0,0,null,null,null,null,null,null
2015,1,1,4,"""DL""",806,"""N3730B""","""SFO""","""MSP""",25,20,-5,18,38,217,230,206,1589,604,6,602,610,8,0,0,null,null,null,null,null,null
2015,1,1,4,"""NK""",612,"""N635NK""","""LAS""","""MSP""",25,19,-6,11,30,181,170,154,1299,504,5,526,509,-17,0,0,null,null,null,null,null,null
2015,1,1,4,"""US""",2013,"""N584UW""","""LAX""","""CLT""",30,44,14,13,57,273,249,228,2125,745,8,803,753,-10,0,0,null,null,null,null,null,null
2015,1,1,4,"""AA""",1112,"""N3LAAA""","""SFO""","""DFW""",30,19,-11,17,36,195,193,173,1464,529,3,545,532,-13,0,0,null,null,null,null,null,null
2015,1,1,4,"""DL""",1173,"""N826DN""","""LAS""","""ATL""",30,33,3,12,45,221,203,186,1747,651,5,711,656,-15,0,0,null,null,null,null,null,null


In [53]:
vuelos_lf.collect()

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
i32,i32,i32,i32,str,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i32,i32,i32,i32,i32
2015,1,1,4,"""AS""",98,"""N407AS""","""ANC""","""SEA""",5,2354,-11,21,15,205,194,169,1448,404,4,430,408,-22,0,0,null,null,null,null,null,null
2015,1,1,4,"""AA""",2336,"""N3KUAA""","""LAX""","""PBI""",10,2,-8,12,14,280,279,263,2330,737,4,750,741,-9,0,0,null,null,null,null,null,null
2015,1,1,4,"""US""",840,"""N171US""","""SFO""","""CLT""",20,18,-2,16,34,286,293,266,2296,800,11,806,811,5,0,0,null,null,null,null,null,null
2015,1,1,4,"""AA""",258,"""N3HYAA""","""LAX""","""MIA""",20,15,-5,15,30,285,281,258,2342,748,8,805,756,-9,0,0,null,null,null,null,null,null
2015,1,1,4,"""AS""",135,"""N527AS""","""SEA""","""ANC""",25,24,-1,11,35,235,215,199,1448,254,5,320,259,-21,0,0,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2015,12,31,4,"""B6""",688,"""N657JB""","""LAX""","""BOS""",2359,2355,-4,22,17,320,298,272,2611,749,4,819,753,-26,0,0,null,null,null,null,null,null
2015,12,31,4,"""B6""",745,"""N828JB""","""JFK""","""PSE""",2359,2355,-4,17,12,227,215,195,1617,427,3,446,430,-16,0,0,null,null,null,null,null,null
2015,12,31,4,"""B6""",1503,"""N913JB""","""JFK""","""SJU""",2359,2350,-9,17,7,221,222,197,1598,424,8,440,432,-8,0,0,null,null,null,null,null,null


In [52]:
vuelos_lf.gather_every(5).collect() ## toma cada enesima fila y nos devuelve otro lazy frame, cada cuantas filas tomar una fila 1 cada 5 filas

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
i32,i32,i32,i32,str,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i32,i32,i32,i32,i32
2015,1,1,4,"""AS""",98,"""N407AS""","""ANC""","""SEA""",5,2354,-11,21,15,205,194,169,1448,404,4,430,408,-22,0,0,null,null,null,null,null,null
2015,1,1,4,"""DL""",806,"""N3730B""","""SFO""","""MSP""",25,20,-5,18,38,217,230,206,1589,604,6,602,610,8,0,0,null,null,null,null,null,null
2015,1,1,4,"""DL""",2336,"""N958DN""","""DEN""","""ATL""",30,24,-6,12,36,173,149,133,1199,449,4,523,453,-30,0,0,null,null,null,null,null,null
2015,1,1,4,"""AS""",108,"""N309AS""","""ANC""","""SEA""",45,41,-4,17,58,204,194,173,1448,451,4,509,455,-14,0,0,null,null,null,null,null,null
2015,1,1,4,"""NK""",520,"""N525NK""","""LAS""","""MCI""",55,120,25,11,131,162,143,128,1139,539,4,537,543,6,0,0,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2015,12,31,4,"""AS""",471,"""N570AS""","""LAX""","""SEA""",2355,2353,-2,17,10,169,152,132,954,222,3,244,225,-19,0,0,null,null,null,null,null,null
2015,12,31,4,"""F9""",682,"""N701FR""","""DEN""","""MCO""",2359,2354,-5,12,6,206,190,170,1546,456,8,525,504,-21,0,0,null,null,null,null,null,null
2015,12,31,4,"""UA""",671,"""N33209""","""PDX""","""IAH""",2359,2352,-7,10,2,245,233,214,1825,536,9,604,545,-19,0,0,null,null,null,null,null,null


In [55]:
vuelos_lf.shift(3, fill_value='prueba').collect()## se desplazo tres filas para abajo,valores nulos, rellenamos los valores con la palabra prueba

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba"""
"""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba"""
"""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba"""
"""2015""","""1""","""1""","""4""","""AS""","""98""","""N407AS""","""ANC""","""SEA""","""5""","""2354""","""-11""","""21""","""15""","""205""","""194""","""169""","""1448""","""404""","""4""","""430""","""408""","""-22""","""0""","""0""",null,null,null,null,null,null
"""2015""","""1""","""1""","""4""","""AA""","""2336""","""N3KUAA""","""LAX""","""PBI""","""10""","""2""","""-8""","""12""","""14""","""280""","""279""","""263""","""2330""","""737""","""4""","""750""","""741""","""-9""","""0""","""0""",null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2015""","""12""","""31""","""4""","""B6""","""802""","""N589JB""","""SLC""","""MCO""","""2359""","""15""","""16""","""14""","""29""","""249""","""250""","""211""","""1931""","""600""","""25""","""608""","""625""","""17""","""0""","""0""",null,"""1""","""0""","""16""","""0""","""0"""
"""2015""","""12""","""31""","""4""","""B6""","""98""","""N607JB""","""DEN""","""JFK""","""2359""","""6""","""7""","""13""","""19""","""211""","""193""","""173""","""1626""","""512""","""7""","""530""","""519""","""-11""","""0""","""0""",null,null,null,null,null,null
"""2015""","""12""","""31""","""4""","""B6""","""66""","""N655JB""","""ABQ""","""JFK""","""2359""","""15""","""16""","""9""","""24""","""227""","""214""","""190""","""1826""","""534""","""15""","""546""","""549""","""3""","""0""","""0""",null,null,null,null,null,null


In [60]:
query = (
    vuelos_lf.with_columns(AIR_TIME_HR = pl.col('AIR_TIME').cast(pl.Float64)/60) # Cast 'AIR_TIME' column to Float64 before division
    .inspect()  ## creo una inspeccion del nodo anterior, luego procede y el select
    .select(
        pl.col('AIR_TIME'),
        pl.col('AIR_TIME_HR') # Use pl.col instead of PL.col
    )
)
query.collect()

shape: (5_819_079, 2)
┌──────────┬─────────────┐
│ AIR_TIME ┆ AIR_TIME_HR │
│ ---      ┆ ---         │
│ i32      ┆ f64         │
╞══════════╪═════════════╡
│ 169      ┆ 2.816667    │
│ 263      ┆ 4.383333    │
│ 266      ┆ 4.433333    │
│ 258      ┆ 4.3         │
│ 199      ┆ 3.316667    │
│ …        ┆ …           │
│ 272      ┆ 4.533333    │
│ 195      ┆ 3.25        │
│ 197      ┆ 3.283333    │
│ 144      ┆ 2.4         │
│ 189      ┆ 3.15        │
└──────────┴─────────────┘


AIR_TIME,AIR_TIME_HR
i32,f64
169,2.816667
263,4.383333
266,4.433333
258,4.3
199,3.316667
…,…
272,4.533333
195,3.25
197,3.283333


In [61]:
query.explain()

'OPAQUE_PYTHON\n   WITH_COLUMNS:\n   [[(col("AIR_TIME").strict_cast(Float64)) / (60.0)].alias("AIR_TIME_HR")] \n    Parquet SCAN [/content/vuelos.parquet]\n    PROJECT 1/31 COLUMNS'